In [214]:
import datetime
import pandas as pd

import pickle

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [215]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [216]:
MODEL_FOLDER = "model"

PROCESSED_DATA_FOLDER = "data/4_all_data_preprocessed"
ISW_DATA_FILE = "all_isw.csv"
WEATHER_EVENTS_DATA_FILE = "features_generated_v1.csv"

OUTPUT_DATA_FILE = "all_features"

tfidf_transformer_model = "tfidf_transformer"
count_vectorizer_model = "count_vectorizer"

tfidf_transformer_version = "v4"
count_vectorizer_version = "v4"

In [217]:
def isNaN(num):
    return num != num

### Load preprocessed data

In [218]:
# Load CSVs
df_isw = pd.read_csv(f"{PROCESSED_DATA_FOLDER}/{ISW_DATA_FILE}", sep=";")
df_weather_events = pd.read_csv(f"{PROCESSED_DATA_FOLDER}/{WEATHER_EVENTS_DATA_FILE}", sep=";")

df_weather_events = df_weather_events.loc[:, ~df_weather_events.columns.str.contains('^Unnamed')]

print(df_isw.shape)

# Load TF-IDF
tfidf = pickle.load(open(f"{MODEL_FOLDER}/{tfidf_transformer_model}_{tfidf_transformer_version}.pkl", "rb"))
cv = pickle.load(open(f"{MODEL_FOLDER}/{count_vectorizer_model}_{count_vectorizer_version}.pkl", "rb"))

(402, 7)


### Prepare weather & events

In [219]:
# Technicaly it's feature engineering

# Replace timestamps with numerical values
df_weather_events.day_sunset = df_weather_events.day_sunset.apply(lambda x: 
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()
)
df_weather_events.day_sunrise = df_weather_events.day_sunrise.apply(lambda x: 
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()
)
df_weather_events.hour_datetime = df_weather_events.hour_datetime.apply(lambda x:
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()//3600
)

# Encode categories
le = preprocessing.LabelEncoder()
df_weather_events.hour_conditions = le.fit_transform(df_weather_events.hour_conditions)
df_weather_events.hour_preciptype = le.fit_transform(df_weather_events.hour_preciptype)

# Create new column with boolean event indicator
df_weather_events["event_indicator"] = df_weather_events.event_hour_level_event_datetimeEpoch.apply(lambda x: not isNaN(x))

# Drop now irrelevant features
features_to_drop = [
    "day_datetimeEpoch",
    "hour_datetimeEpoch",
    "event_all_region",
    "event_clean_end",
    "event_intersection_alarm_id",
    "event_start_time",
    "event_end_time",
    "event_start_hour",
    "event_end_hour",
    "event_day_date",
    "event_start_hour_datetimeEpoch",
    "event_end_hour_datetimeEpoch",	
    "event_hour_level_event_time",
    "event_hour_level_event_datetimeEpoch",
    "event_region_title",
    "event_region_city",
    "event_start",
    "event_end",
    "city",
    "region",
    "center_city_ua",
    "center_city_en",
    "region_alt"
]
df_weather_events = df_weather_events.drop(features_to_drop, axis=1)

# Convert everything to float
df_weather_events.hour_preciptype = df_weather_events.hour_preciptype.astype(float)
df_weather_events.hour_conditions = df_weather_events.hour_conditions.astype(float)
df_weather_events.region_id = df_weather_events.region_id.astype(float)
df_weather_events.event_alarms_past_24 = df_weather_events.event_alarms_past_24.astype(float)
df_weather_events.event_simultaneous_alarms = df_weather_events.event_simultaneous_alarms.astype(float)

df_weather_events.event_holiday_is_near = df_weather_events.event_holiday_is_near.apply(lambda x: 1.0 if x and not isNaN(x) else 0.0)

# Fill NAN
df_weather_events.fillna(0, inplace=True)

print(df_weather_events.shape)
print(df_weather_events.dtypes)
df_weather_events.sample(10)

(195084, 41)
city_resolvedAddress            object
day_datetime                    object
day_tempmax                    float64
day_tempmin                    float64
day_temp                       float64
day_dew                        float64
day_humidity                   float64
day_precip                     float64
day_precipcover                float64
day_solarradiation             float64
day_solarenergy                float64
day_uvindex                    float64
day_sunrise                    float64
day_sunset                     float64
day_moonphase                  float64
hour_datetime                  float64
hour_temp                      float64
hour_humidity                  float64
hour_dew                       float64
hour_precip                    float64
hour_precipprob                float64
hour_snow                      float64
hour_snowdepth                 float64
hour_preciptype                float64
hour_windgust                  float64
hour_windspe

,city_resolvedAddress,day_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,day_sunrise,day_sunset,day_moonphase,hour_datetime,hour_temp,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_severerisk,hour_conditions,region_id,event_alarms_past_24,event_simultaneous_alarms,event_hours_from_last_alarm,event_holiday_is_near,event_indicator
145990,"Полтава, Україна",2022-02-28,1.4,-3.7,-1.0,-4.4,77.5,0.0,0.00,127.4,11.0,4.0,23222.0,62550.0,0.90,17.0,0.2,83.28,-2.3,0.0,0.0,0.0,0.0,5.0,34.9,14.4,20.0,1028.3,10.0,100.0,175.0,0.6,2.0,10.0,3.0,16.0,3.0,0.0,3.260833,0.0,False
30684,"Київ, Україна",2022-06-15,22.3,12.9,17.7,6.0,48.1,0.0,0.00,265.6,23.0,9.0,17156.0,76266.0,0.52,3.0,14.4,57.83,6.2,0.0,0.0,0.0,0.0,5.0,19.8,3.6,290.0,1017.2,20.0,100.0,0.0,0.0,0.0,10.0,3.0,10.0,6.0,3.0,1.480833,0.0,True
9197,"Кропивницький, Україна",2022-04-09,22.2,4.3,11.7,4.5,65.0,0.4,4.17,188.8,16.3,6.0,22380.0,70385.0,0.26,5.0,4.5,88.72,2.8,0.0,0.0,0.0,0.0,5.0,7.2,4.0,164.1,1007.0,24.1,64.9,0.0,0.0,0.0,10.0,4.0,11.0,8.0,13.0,5.966389,1.0,False
88784,"Хмельницька область, Україна",2022-03-10,-3.1,-6.6,-4.4,-9.6,68.5,0.3,4.17,144.5,12.3,4.0,23794.0,65347.0,0.24,11.0,-3.4,69.93,-8.1,0.0,0.0,0.1,0.4,4.0,39.6,25.2,50.0,1030.9,10.0,90.0,322.0,1.2,3.0,10.0,4.0,22.0,5.0,2.0,6.507778,0.0,False
91188,"Хмельницька область, Україна",2022-06-06,23.2,10.6,18.0,11.6,69.7,0.0,0.00,269.5,23.3,9.0,18473.0,76451.0,0.22,7.0,11.3,98.03,11.0,0.0,0.0,0.0,0.0,5.0,3.6,1.4,0.5,1020.0,24.1,63.0,132.0,0.5,1.0,10.0,4.0,22.0,1.0,1.0,23.535278,0.0,False
95359,"Хмельницька область, Україна",2022-11-26,0.4,-0.8,-0.3,-1.2,93.8,0.1,4.17,58.2,5.1,3.0,27564.0,58739.0,0.07,1.0,-0.8,94.31,-1.6,0.0,0.0,0.0,7.2,4.0,19.1,9.7,328.3,1022.0,2.3,100.0,0.0,0.0,0.0,10.0,3.0,22.0,0.0,0.0,9.775278,0.0,False
70431,"Миколаїв, Україна",2022-03-22,11.9,3.0,7.0,-1.6,57.0,0.0,0.00,172.7,15.0,7.0,21091.0,65237.0,0.65,15.0,11.7,29.58,-5.5,0.0,0.0,0.0,0.0,5.0,44.6,20.9,64.4,1032.0,24.1,0.0,559.0,2.0,6.0,10.0,0.0,14.0,10.0,3.0,2.300278,0.0,True
138149,"Тернопіль, Україна",2022-03-07,-1.3,-3.4,-2.2,-5.7,77.1,0.3,12.50,146.0,11.5,4.0,24507.0,65388.0,0.14,14.0,-2.0,70.76,-6.6,0.1,100.0,0.0,0.0,4.0,33.8,17.6,325.3,1017.0,18.2,100.0,358.0,1.3,4.0,10.0,9.0,19.0,0.0,13.0,15.100556,0.0,False
31585,"Київ, Україна",2022-07-16,24.1,13.2,19.5,10.4,59.5,0.0,0.00,314.8,27.2,9.0,18262.0,75778.0,0.57,19.0,22.8,36.36,7.1,0.0,0.0,0.0,0.0,5.0,22.7,7.9,286.9,1015.0,24.1,64.4,298.0,1.1,3.0,10.0,4.0,10.0,6.0,3.0,0.899722,0.0,True
38117,"Херсон, Україна",2022-04-20,10.2,4.8,7.4,3.3,75.3,0.4,4.17,142.7,12.3,7.0,21225.0,71049.0,0.63,5.0,5.8,85.16,3.5,0.0,0.0,0.0,0.0,3.0,12.2,8.3,217.8,1010.0,21.6,90.5,0.0,0.0,0.0,10.0,3.0,21.0,0.0,0.0,4.274722,0.0,False


### Calculate TF-IDF matrix

In [220]:
# Calculate word count vector for each report (save as matrix)
word_count_vector = cv.transform(df_isw["lemm"].values.astype("U"))
# print("Word count vector:\n", word_count_vector)

# Calculate global tfidf vectors matrix
tfidf_vector = tfidf.transform(word_count_vector)

# transform tfidf matrix to sparse dataframe
df_tfidf_vector = pd.DataFrame.sparse.from_spmatrix(tfidf_vector, columns=cv.get_feature_names_out())
df_tfidf_vector = pd.concat([df_isw.date_tomorrow_datetime, df_tfidf_vector], axis=1)

print(df_tfidf_vector.shape)

(402, 719)


### Merge final dataset

In [221]:
# Merge weather events dataset with yesterday report tfidf matrix (takes 2m to execute)
df_final = df_weather_events.merge(df_tfidf_vector.add_prefix("isw_"),
                                    how="inner",
                                    left_on="day_datetime",
                                    right_on="isw_date_tomorrow_datetime")

# Normalize
scaler = StandardScaler()
df_float_values = df_final.drop(['event_indicator', 'city_resolvedAddress', 'day_datetime', 'isw_date_tomorrow_datetime'], axis=1)
float_values_scaled = scaler.fit_transform(df_float_values.to_numpy())
df_fload_values_scaled = pd.DataFrame(float_values_scaled, columns=df_float_values.columns)
df_final = pd.concat([df_final[['event_indicator', 'city_resolvedAddress', 'day_datetime', 'isw_date_tomorrow_datetime']], df_fload_values_scaled], axis=1)

print(df_final.shape)
df_final.sample(5)

(192867, 760)


,event_indicator,city_resolvedAddress,day_datetime,isw_date_tomorrow_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,day_sunrise,day_sunset,day_moonphase,hour_datetime,hour_temp,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_severerisk,hour_conditions,region_id,event_alarms_past_24,event_simultaneous_alarms,event_hours_from_last_alarm,event_holiday_is_near,isw_10km,isw_12km,isw_15km,isw_1st,isw_20km,isw_30km,isw_5km,isw_6km,isw_abandon,isw_abil,isw_abl,isw_access,isw_accus,isw_achiev,isw_across,isw_action,isw_activ,isw_ad,isw_addit,isw_address,isw_administr,isw_advanc,isw_advisor,isw_affect,isw_affili,isw_aim,isw_air,isw_airborn,isw_aircraft,isw_airstrik,isw_alexand,isw_allow,isw_almost,isw_along,isw_alongsid,isw_alreadi,isw_also,isw_although,isw_ammunit,isw_among,isw_amplifi,isw_annex,isw_announc,isw_anoth,isw_anti,isw_appar,isw_appeal,isw_appear,isw_appoint,isw_approxim,isw_april,isw_area,isw_arm,isw_armi,isw_armor,isw_around,isw_arrang,isw_arriv,isw_artilleri,isw_assault,isw_asset,isw_attempt,isw_author,isw_avail,isw_avdiivka,isw_avoid,isw_away,isw_axi,isw_back,isw_bakhmut,isw_bank,isw_base,isw_basi,isw_battalion,isw_battl,isw_becom,isw_began,isw_begin,isw_belaru,isw_belarusian,isw_belgorod,isw_bilohorivka,isw_blame,isw_border,isw_break,isw_bridg,isw_brigad,isw_build,isw_call,isw_campaign,isw_capabl,isw_captur,isw_carri,isw_case,isw_casualti,isw_cauldron,isw_caus,isw_center,isw_central,isw_challeng,isw_chang,isw_channel,isw_chechen,isw_chief,isw_child,isw_choos,isw_citizen,isw_civilian,isw_classifi,isw_clear,isw_click,isw_close,isw_colonel,isw_combat,isw_combin,isw_come,isw_command,isw_comment,isw_commerci,isw_commit,isw_commun,isw_compani,isw_complet,isw_compris,isw_concentr,isw_concern,isw_condemn,isw_condit,isw_confirm,isw_conflict,isw_connect,isw_conscript,isw_consid,isw_consist,isw_consolid,isw_construct,isw_contact,isw_contract,isw_control,isw_convent,isw_corp,isw_could,isw_council,isw_counter,isw_counterattack,isw_counteroffens,isw_countri,isw_cours,isw_cover,isw_creat,isw_crime,isw_crimea,isw_crimin,isw_critic,isw_cross,isw_current,isw_cut,isw_daili,isw_damag,isw_data,isw_date,isw_day,isw_decemb,isw_decis,isw_defend,isw_defens,isw_degrad,isw_demand,isw_demonstr,isw_deni,isw_deploy,isw_depot,isw_deputi,isw_describ,isw_despit,isw_destroy,isw_detail,isw_develop,isw_differ,isw_direct,isw_directli,isw_director,isw_discus,isw_disrupt,isw_district,isw_divis,isw_dmitri,isw_dnipro,isw_dnipropetrovsk,isw_dnr,isw_domest,isw_donba,isw_draw,isw_drive,isw_drone,isw_due,isw_duma,isw_earli,isw_east,isw_eastern,isw_econom,isw_effect,isw_eight,isw_eighteen,isw_eighti,isw_either,isw_element,isw_eleven,isw_emphas,isw_encircl,isw_end,isw_endnot,isw_energi,isw_enerhodar,isw_enforc,isw_engag,isw_enter,isw_entir,isw_entireti,isw_equip,isw_establish,isw_evacu,isw_evalu,isw_even,isw_evid,isw_exist,isw_expand,isw_expect,isw_explos,isw_express,isw_extens,isw_face,isw_facil,isw_fail,isw_failur,isw_fall,isw_fals,isw_famili,isw_far,isw_februari,isw_feder,isw_fifteen,isw_fifti,isw_fight,isw_fighter,isw_financi,isw_fire,isw_first,isw_five,isw_focu,isw_focus,isw_follow,isw_footag,isw_forcibl,isw_forecast,isw_foreign,isw_form,isw_format,isw_former,isw_forti,isw_fortif,isw_four,isw_fourteen,isw_front,isw_frontlin,isw_full,isw_fulli,isw_futur,isw_gain,isw_geneva,isw_geoloc,isw_geospati,isw_given,isw_gloc,isw_govern,isw_governor,isw_ground,isw_group,isw_grow,isw_guard,isw_gur,isw_haidai,isw_head,isw_heavi,isw_heavili,isw_held,isw_high,isw_highli,isw_highway,isw_himar,isw_hold,isw_hope,isw_hour,isw_howev,isw_hulyaipol,isw_human,isw_hundr,isw_hundred,isw_identifi,isw_imageri,isw_immedi,isw_immin,isw_import,isw_improv,isw_includ,isw_increas,isw_increasingli,isw_independ,isw_indic,isw_

In [222]:
# Save final dataset
with open(f"{PROCESSED_DATA_FOLDER}/{OUTPUT_DATA_FILE}.pkl", 'wb') as handle:
    pickle.dump(df_final, handle)